# Format data

Combines the rotation and play-by-play data into lineups and points scored over time. Note: some points are scored during free-throws between substitutions and, for simplicity, these points are omitted. So, some games the lineup data total points might not be the true final score.

In [ ]:
import os
from glob import glob
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
from multiprocessing import Pool

In [ ]:
# first season with play-by-play data is 1996-97
FIRST_SEASON = 1996
LAST_SEASON = 2021
ALL_SEASONS = [f'{x}-{str(x+1)[-2:]}' for x in range(FIRST_SEASON, LAST_SEASON+1)]

In [ ]:
def format_season(season, 
                  base_folder_pbp='data/nba-api/pbp', 
                  base_folder_rot='data/nba-api/rotation', 
                  base_save_folder='data/lineup', 
                  parallel=False):
    
    print(season)
    
    # create save folder
    season_save_folder = f'{base_save_folder}/{season}'
    if not os.path.exists(season_save_folder):
        os.makedirs(season_save_folder)
        
    # list all play-by-play and rotation data for season
    season_files = pd.DataFrame()
    season_files['pbp_file'] = glob(f'{base_folder_pbp}/{season}/*')
    season_files['rot_file'] = season_files.pbp_file.str.replace(base_folder_pbp, base_folder_rot)  

    if parallel:
    
        pbar = tqdm(total=len(season_files))
        pool = Pool()
        for _, row in season_files.iterrows():
            pool.apply_async(_format_game, 
                             args=(row, season_save_folder), 
                             callback=lambda x: pbar.update())
        pool.close()
        pool.join()
        pbar.close()
        
    else:
        
        for _, row in tqdm(season_files.iterrows(), total=len(season_files)):
            try:
                _format_game(row, season_save_folder)
            except:
                continue
        
    
def _format_game(row, season_save_folder):
    
    # get game id and save path
    game_id = row['pbp_file'].split('/')[-1].split('.')[0]
    save_file = f'{season_save_folder}/{game_id}.csv'
    # if save path already exists, skip formatting
    if os.path.exists(save_file):
        return

    # format play by play to get times and scores, plus team ids
    df_pbp, home_id, visit_id = _format_pbp_file(row['pbp_file'])
    # format rotations to get lineups at all times
    df_rot = _format_rot_file(row['rot_file'], home_id, visit_id)
    # merge dataframes on time
    df_lineup = df_rot.merge(df_pbp, on='time', how='left')

    # group each streak of lineups
    lineup_groups = []
    for t, (k,g) in enumerate((itertools.groupby
                               (df_lineup[['h1', 'h2', 'h3', 'h4', 'h5', 
                                           'v1', 'v2', 'v3', 'v4', 'v5']].values.tolist()))):
        lineup_groups.extend([t]*len(list(g)))
    df_lineup['group'] = lineup_groups
    
    # get players, times, and points for each streak group
    data = []
    for k,g in df_lineup.groupby('group'):
        players = g[['h1', 'h2', 'h3', 'h4', 'h5', 'v1', 'v2', 'v3', 'v4', 'v5']].iloc[0]
        start_time = g.time.min()
        end_time = g.time.max()
        home_points = g.home_score.max() - g.home_score.min()
        visit_points = g.visit_score.max() - g.visit_score.min()
        data.append(players.to_list() + [start_time, end_time, home_points, visit_points])
    data = pd.DataFrame(data, columns=(players.index.to_list() + 
                                       ['start', 'end', 'home_points', 'visit_points']))
    data = data.astype(int)

    data.to_csv(save_file, index=False)  

def _format_pbp_file(filename):
    
    df_pbp = pd.read_csv(filename)

    # get home and away team ids
    vals, counts = np.unique(df_pbp.loc[~df_pbp.HOMEDESCRIPTION.isna(), 'PLAYER1_TEAM_ID'], return_counts=True)
    home_id = int(vals[np.argmax(counts)])
    vals, counts = np.unique(df_pbp.loc[~df_pbp.VISITORDESCRIPTION.isna(), 'PLAYER1_TEAM_ID'], return_counts=True)
    visit_id = int(vals[np.argmax(counts)])    
    
    # add team scores
    df_pbp.loc[df_pbp.EVENTNUM == df_pbp.EVENTNUM.min(), 'SCORE'] = '0 - 0'
    df_pbp['home_score'] = df_pbp.SCORE.str.split('-').str.get(1).str.strip().ffill().astype(int)
    df_pbp['visit_score'] = df_pbp.SCORE.str.split('-').str.get(0).str.strip().ffill().astype(int)
    
    # add elapse time
    seconds_temp = df_pbp.PCTIMESTRING.str.split(':', expand=True).astype(np.int64).apply(lambda x: x[0]*60 + x[1], axis=1)
    df_pbp['time'] = np.where(df_pbp.PERIOD < 5, 
                              720 * df_pbp.PERIOD - seconds_temp,  
                              300 * df_pbp.PERIOD - seconds_temp + 1680)
    
    # clean up
    df_pbp = df_pbp.groupby('time')[['visit_score', 'home_score']].max().reset_index()
    df_pbp['visit_score'] = df_pbp.visit_score.ffill().astype(int) 
    df_pbp['home_score'] = df_pbp.home_score.ffill().astype(int)

    return df_pbp, home_id, visit_id


def _format_rot_file(filename, home_id, visit_id):
    
    df_rot = pd.read_csv(filename)

    # convert time to seconds and calculate time ranges
    df_rot['IN_TIME_REAL'] = (df_rot.IN_TIME_REAL / 10).astype(int)
    df_rot['OUT_TIME_REAL'] = (df_rot.OUT_TIME_REAL / 10).astype(int)
    df_rot['time'] = df_rot.apply(lambda x: np.arange(x['IN_TIME_REAL'], x['OUT_TIME_REAL']), axis=1)    
    
    # expand time ranges
    df_lineup = pd.DataFrame()
    df_lineup['time'] = np.concatenate(df_rot.time)
    df_lineup['person_id'] = np.concatenate(df_rot.apply(lambda x: [x['PERSON_ID']] * len(x['time']), axis=1))
    df_lineup['team_id'] = np.concatenate(df_rot.apply(lambda x: [x['TEAM_ID']] * len(x['time']), axis=1))
    
    # split into home and away and aggregate lineups
    home_lineup = df_lineup[df_lineup.team_id==home_id]
    visit_lineup = df_lineup[df_lineup.team_id==visit_id]
    
    home_lineup = home_lineup.groupby('time').person_id.apply(lambda x: tuple(sorted(x))).reset_index()
    visit_lineup = visit_lineup.groupby('time').person_id.apply(lambda x: tuple(sorted(x))).reset_index()
    
    home_lineup.rename(columns={'person_id' : 'home_lineup'}, inplace=True)
    visit_lineup.rename(columns={'person_id' : 'visit_lineup'}, inplace=True)
    
    df_lineup = home_lineup.merge(visit_lineup, on='time')
    
    # expand lineup tuples to columns
    df_lineup[['h1', 'h2', 'h3', 'h4', 'h5']] = pd.DataFrame(df_lineup.home_lineup.to_list())
    df_lineup[['v1', 'v2', 'v3', 'v4', 'v5']] = pd.DataFrame(df_lineup.visit_lineup.to_list())
    # remove lineup tuples
    df_lineup.drop(columns=['home_lineup', 'visit_lineup'], inplace=True)
    
    return df_lineup

In [ ]:
for season in ALL_SEASONS:
    format_season(season)